# 1. Import of input data

In [ ]:
import nbimporter
import sys
import os
from pathlib import Path

In [ ]:
#Import libraries
try:
    current_dir = RUN_DIR
    print(f"📂 GeothermalPlant.ipynb executed by RUN.ipynb — using directory: {current_dir}")
except NameError:
    current_dir = Path().resolve()
    print(f"📁 GeothermalPlant.ipynb executed standalone — using directory: {current_dir}")

Libraries_path = (current_dir / "Libraries.ipynb").resolve()

# If not found, try in the parent levels
if not Libraries_path.exists():
    Libraries_path = (current_dir.parent / "Libraries.ipynb").resolve()

# Run the file
if Libraries_path.exists():
    print(f"📘 Importing libraries from: {Libraries_path}")
    get_ipython().run_line_magic('run', f'"{Libraries_path}"')
else:
    print("⚠️ Libraries.ipynb not found! Some functions may not be available.")

In [ ]:
energysystem = pd.read_excel(energysystem_path, sheet_name = 'Geothermal plant')

# 2. Input parameters

In [ ]:
Power = energysystem['Thermal capacity [kW]']
EnergyProduced = energysystem['Thermal energy produced [kWh/year]']
EnergyInputVapour = energysystem['Energy input vapour [kWh/year]'] 
SupplyTemperatureVapour = energysystem['Supply temperature vapour [°C]']  
ReturnTemperatureVapour = energysystem['Return temperature vapour [°C]']  

In [ ]:
Lifespan = 50 #years
HRXLifespan = 15 #years

In [ ]:
massHRXGeothermal = Power * (12 / 450) # Source: Nordictec (450 kW - weight: 12 kg)

# 3. Life cycle inventories

## 3.1 Module A1-3

In [ ]:
Geothermal_lowalloyedsteel = 223000 * (Power / (5500 * 6.4) ) 
Geothermal_stainlessSteel = massHRXGeothermal
massDeepwell = Geothermal_lowalloyedsteel
massHRX = Geothermal_stainlessSteel
mass = Geothermal_lowalloyedsteel + massHRXGeothermal

## 3.2 Module A4

In [ ]:
GeothermalDeepwell_Truk_16_32 = ((500 + 200)*(Geothermal_lowalloyedsteel)/1000)
GeothermalDeepwell_Light_Truk = (50*(Geothermal_lowalloyedsteel)/1000)

GeothermalHRX_Truk_16_32 = ((500 + 200)*(massHRXGeothermal)/1000)
GeothermalHRX_Light_Truk = (50*(massHRXGeothermal)/1000)

## 3.3 Module A5

In [ ]:
Geothermal_deepwell = 32000 * (Power / (5500 * 6.4) ) # 5.5 MWel (Ecoinvent)
Geothermal_dieselA5 = np.full(len(Power), 5900)
GeothermalDeepwell_Light_TrukA5 = (160 * (Geothermal_lowalloyedsteel) / 1000)
GeothermalDeepwell_Truk_16_32A5 = (180 * (Geothermal_lowalloyedsteel) / 1000)

GeothermalHRX_Light_TrukA5 = (160 * (massHRXGeothermal) / 1000)
GeothermalHRX_Truk_16_32A5 = (180 * (massHRXGeothermal) / 1000)

## 3.4 Module B2

In [ ]:
Geothermal_Light_Truk_B2_HRX = np.array(50 * (HRXLifespan / 2) * 0.01 * (massHRXGeothermal / 1000))  # Light_Truk_B2 come array
Geothermal_Light_Truk_B2_well = np.array(50 * (Lifespan / 2) * 0.01 * (Geothermal_lowalloyedsteel / 1000))  # Light_Truk_B2 come array

## 3.5 Module B4

In [ ]:
DeepwellNumberReplacements = (np.ceil(np.divide(ReqSL, Lifespan) - 1))
HRXNumberReplacements = (np.ceil(np.divide(ReqSL, HRXLifespan) - 1))

SFwell = (RSP / ReqSL)  # Scaling factor
SFHRX = (RSP / ReqSL)  # Scaling factor

## 3.6 Module B6

In [ ]:
Geothermal_usephasestimulation = 40000 * (Power / (5500 * 6.4)) * RSP #cubic meter (This is a chemical technique for incresing the vapour flow rate)

## 3.7 Module C2

In [ ]:
GeothermalDeepwell_Transport_to_treatment_plant = 50 * (massDeepwell/1000)
GeothermalHRX_Transport_to_treatment_plant = 50 * (massHRX/1000)

## 3.8 Module C3

In [ ]:
Geothermal_EOL_lowalloyedsteel = massDeepwell
Geothermal_EOL_stainlessSteel = massHRX

## 3.9 Module C4

In [ ]:
Geothermal_EOL_landfill_lowalloyedsteel = Geothermal_EOL_lowalloyedsteel * LandfillMetals
Geothermal_EOL_landfill_stainlessSteel = Geothermal_EOL_stainlessSteel * LandfillMetals2

## 3.10 Module D1 (related to the export of secondary materials)

In [ ]:
Geothermal_EOL_recycling_lowalloyedsteel = (RecyclingMetals * RecyclingEfficiencyMetals * Geothermal_EOL_lowalloyedsteel) - (RecycledContentMetals * Geothermal_EOL_lowalloyedsteel)
Geothermal_EOL_primary_lowalloyedsteel = ((RecycledContentMetals * Geothermal_EOL_lowalloyedsteel) - (RecyclingMetals * RecyclingEfficiencyMetals * Geothermal_EOL_lowalloyedsteel)) * SRmetals

Geothermal_EOL_recycling_stainlessSteel = (RecyclingMetals2 * RecyclingEfficiencyMetals * Geothermal_EOL_stainlessSteel) - (RecycledContentMetals * Geothermal_EOL_stainlessSteel)
Geothermal_EOL_primary_stainlessSteel = ((RecycledContentMetals * Geothermal_EOL_stainlessSteel) - (RecyclingMetals2 * RecyclingEfficiencyMetals * Geothermal_EOL_stainlessSteel)) * SRmetals


# 4. Data for export

In [ ]:
ThermalEnergyProduced_Geothermal = np.array([EnergyProduced * RSP])